In [1]:
import csv
import itertools
import re
from string import Template
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
MYANMMAR_CHARITIES_URL ='https://www.lrcmyanmar.org/ngo-database/?type1=Community%20Based%20Organization#'
MYANMMAR_CSV_DUMP_PATH = './NGO_vietnam.csv'

In [3]:
def convert(text, string_append):
    #combine the input text with a string to get a website id
    to_combine = text.split()
    for ix, item in enumerate(to_combine):
        if ix != len(to_combine)-1:
            to_combine[ix] = item+'%20'
    return  'https://www.lrcmyanmar.org' + string_append +''.join(to_combine)

def gen_websitecontainer_to_scrap(container):
    #return a list of website to scrap
    link_to_scrap= []
    for items in container:
        #import pdb;pdb.set_trace()
        try: 
            #check if the list is more than one element long
            list_iterate = items.text.split(',')
            for item in list_iterate:
                link_to_scrap.append(convert(item,'/ngo-list/?sector='))
        except:
            link_to_scrap.append(convert(items.text,'/ngo-list/?sector='))
    return link_to_scrap

def gen_website_to_scrape(list_of_website_container):
    website_list = []
    for website_container in list_of_website_container:
        request1 = requests.get(website_container)
        soup1 = BeautifulSoup(request1.text, 'html.parser')
        test_list = soup1.find_all('td')
        for item in test_list:
            website_list.append(convert(item.text,'/ngo-info/?$name='))
    return website_list

def scrape_site(site_name):

    def convert_to_dataframe (X,y):
        #X is the keys,while y is the values
        dicte = dict(zip(X,y))
        df = pd.DataFrame(dicte,index=[0])
        return df

    table_index=[]
    table_val = []
    #import pdb;pdb.set_trace()
    request = requests.get(site_name)
    soup = BeautifulSoup(request.text, 'html.parser')
    temp_table_index = soup.find_all('td',class_ = False)
    table_index = [x.text for x  in temp_table_index]
    temp_table_val = soup.find_all('td', {'class':'tr-main'})
    table_val = [x.text for x in temp_table_val]
    return convert_to_dataframe(table_index[:-2],table_val[:-2])

def scrape_all (website_list):
    for ix, website in enumerate(website_list):
        #import pdb;pdb.set_trace()
        if ix ==0:
            df = scrape_site(website)
        else:
            df = pd.concat([df,scrape_site(website)])
        print('site {} out of {} is completed'.format(ix, len(website_list)))
    return df



def data_clean(df):
    #data cleansing
    f = df['Organization Name'].isna()
    #drop those without organization name
    df = df.loc[~f]
    rename_col = {
        'Organization Name':'Name',
        'Extra Information ':'description',
        'Website':'website',
        'Sectors':'cause_area',
        'Contact Address':'address',
        'States or Regions ': "city",
        'Contact Phone':'Contact_number',
        'Contact Email':'email',
        'Contact person':'contact_person' 
    }
    keep = ['Name', 'description', 'website', 'cause_area', 'address', 'city', 'Contact_number', 'email', 'contact_person']
    df = df.rename(columns=rename_col)
    df = df[keep]
    df = df.replace('',np.nan)
    return df

In [4]:
request = requests.get(MYANMMAR_CHARITIES_URL)
soup = BeautifulSoup(request.text, 'html.parser')

#find all the website links under the different sectors and geography
charities_container_sectors = soup.find_all("td", {'class':"sector1"})
charities_container_geography = soup.find_all('td', {'class':'state1'})

#generate the list of webcontainers
list_of_website_container = gen_websitecontainer_to_scrap(charities_container_sectors)+gen_websitecontainer_to_scrap(charities_container_geography)

#generate the list of websites
list_of_website = gen_website_to_scrape(list_of_website_container)

#deduplicate identical websites
list_of_website = list(set(list_of_website))

#scrape all websites
df = scrape_all(list_of_website)

#clean and save output
df = data_clean(df)

df.to_csv(MYANMMAR_CSV_DUMP_PATH,na_rep='Nan',index=False)

site 0 out of 158 is completed
site 1 out of 158 is completed
site 2 out of 158 is completed
site 3 out of 158 is completed
site 4 out of 158 is completed
site 5 out of 158 is completed
site 6 out of 158 is completed
site 7 out of 158 is completed
site 8 out of 158 is completed
site 9 out of 158 is completed
site 10 out of 158 is completed
site 11 out of 158 is completed
site 12 out of 158 is completed
site 13 out of 158 is completed
site 14 out of 158 is completed
site 15 out of 158 is completed
site 16 out of 158 is completed
site 17 out of 158 is completed
site 18 out of 158 is completed
site 19 out of 158 is completed
site 20 out of 158 is completed
site 21 out of 158 is completed
site 22 out of 158 is completed
site 23 out of 158 is completed
site 24 out of 158 is completed
site 25 out of 158 is completed
site 26 out of 158 is completed
site 27 out of 158 is completed
site 28 out of 158 is completed
site 29 out of 158 is completed
site 30 out of 158 is completed
site 31 out of 158

In [9]:
df.isnull().sum()

Name                0
description       132
website           115
cause_area         14
address           131
city               36
Contact_number     49
email              93
contact_person     78
dtype: int64